# Digit Recognition using Artificial Neural Networks

The purpose of this notebook is to develop artificial neural networks that correctly recognizes hand-drawn digits from 0-9. A successful algorithm could be used to recognize hand written addresses for sorting or correctly identifying forms based upon their name.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

#import sklearn
#import tensorflow

## Ingestion

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/jhancuch/mnist-ann/main/data/train.csv')

In [3]:
test = pd.read_csv('https://raw.githubusercontent.com/jhancuch/mnist-ann/main/data/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
train.columns

In [ ]:
test.columns